In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 21.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,790 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubun

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd

In [ ]:
# Selenium Options
options = webdriver.ChromeOptions()
options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize WebDriver
driver = webdriver.Chrome(options=options)

# Dictionary to store medicine names
medicin = {}
li = []
MediName=[]

# Scraping drug names
for i in "abcdefghijklmnopqrstuvwxyz":
    url = f"https://www.drugs.com/alpha/{i}.html"
    driver.get(url)

    sleep(1)  # Allow time for the page to load

    # Find elements containing medicine names
    med = driver.find_elements(By.CLASS_NAME, "ddc-list-column-2")

    if med:
        medi = med[0].text.split("\n")
        for j in medi:
          MediName.append(j)
          formatted_name = j.lower().replace(" ", "-")
          li.append(formatted_name)
        medicin[i] = medi
    else:
        medicin[i] = []
# Close and reopen the driver to avoid issues
driver.quit()

In [ ]:
# Reinitialize WebDriver
driver = webdriver.Chrome(options=options)

# Scraping drug images

imgLink=[]
for i in li:
    drug_url = f"https://www.drugs.com/{i}.html"
    driver.get(drug_url)
    sleep(1)

    img = driver.find_elements(By.XPATH, "//*[@id='drug-imprint-primary']/img")

    if img:
        image_links = img[0].get_attribute('src')
        imgLink.append(image_links)
        #print(f"{i}: {image_links}")
    else:
        imgLink.append(None)

# Close the driver
driver.quit()

In [ ]:
data = pd.DataFrame(list(zip(MediName, imgLink)), columns=['Medicine', 'Image_Link'])

In [ ]:
data.head()

,Medicine,Image_Link
0,Abecma,https://www.drugs.com/images/pills/custom/pill...
1,Abilify,https://www.drugs.com/images/pills/nlm/59148-0...
2,Abilify Asimtufii,None
3,Abilify Maintena,https://www.drugs.com/images/pills/custom/pill...
4,Abiraterone,https://www.drugs.com/images/pills/fio/NDZ0030...


In [ ]:
data.to_csv("medical_info.csv",index = False)